<h2>
 Write a Pyspark code to list employee name along with their manager and senior manager name.
Senior manager is manager's manager.
</h2>

In [6]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Manager-hierarchy').getOrCreate()

In [38]:
from pyspark.sql.functions import to_timestamp
data=[(1, 'Ankit', 100,10000, 4, 39), 
(2, 'Mohit', 100, 15000, 5, 48),
    (3, 'Vikas', 100, 12000,4,37),
     (4, 'Rohit', 100, 14000, 2, 16),
      (5, 'Mudit', 200, 20000, 6,55),
      (6, 'Agam', 200, 12000,2, 14),
       (7, 'Sanjay', 200, 9000, 2,13),
      (8, 'Ashish', 200,5000,2,12),
      (9, 'Mukesh',300,6000,6,51),
      (10, 'Rakesh',500,7000,6,50)
     ]

schema="""
 emp_id int,
emp_name string,
department_id int,
salary int,
manager_id int,
emp_age int
"""
df=spark.createDataFrame(data,schema)
df.show()

+------+--------+-------------+------+----------+-------+
|emp_id|emp_name|department_id|salary|manager_id|emp_age|
+------+--------+-------------+------+----------+-------+
|     1|   Ankit|          100| 10000|         4|     39|
|     2|   Mohit|          100| 15000|         5|     48|
|     3|   Vikas|          100| 12000|         4|     37|
|     4|   Rohit|          100| 14000|         2|     16|
|     5|   Mudit|          200| 20000|         6|     55|
|     6|    Agam|          200| 12000|         2|     14|
|     7|  Sanjay|          200|  9000|         2|     13|
|     8|  Ashish|          200|  5000|         2|     12|
|     9|  Mukesh|          300|  6000|         6|     51|
|    10|  Rakesh|          500|  7000|         6|     50|
+------+--------+-------------+------+----------+-------+



In [39]:
from pyspark.sql.functions import col
manager_df=df.alias('a').join(df.alias('b'),\
                              col('a.manager_id')==col('b.emp_id'),how='inner').\
select(col('a.emp_id').alias('emp_id'),col('a.emp_name').alias('emp_name'),\
col('b.emp_id').alias("manager_id"),\
col('b.emp_name').alias('manager_name'),col('b.manager_id').alias('grand_manager_id'))
manager_df.show()


+------+--------+----------+------------+----------------+
|emp_id|emp_name|manager_id|manager_name|grand_manager_id|
+------+--------+----------+------------+----------------+
|     4|   Rohit|         2|       Mohit|               5|
|     6|    Agam|         2|       Mohit|               5|
|     7|  Sanjay|         2|       Mohit|               5|
|     8|  Ashish|         2|       Mohit|               5|
|     1|   Ankit|         4|       Rohit|               2|
|     3|   Vikas|         4|       Rohit|               2|
|     2|   Mohit|         5|       Mudit|               6|
|     5|   Mudit|         6|        Agam|               2|
|     9|  Mukesh|         6|        Agam|               2|
|    10|  Rakesh|         6|        Agam|               2|
+------+--------+----------+------------+----------------+



In [40]:
final_df=manager_df.alias('c').join(df.alias('d'),col('c.grand_manager_id')==col('d.emp_id'),how='inner')\
.select(col('c.emp_id'),col('c.emp_name'),col('c.manager_name'),col('d.emp_name').alias('grand_manager_name')).orderBy(col('c.emp_id'))
final_df.show()

+------+--------+------------+------------------+
|emp_id|emp_name|manager_name|grand_manager_name|
+------+--------+------------+------------------+
|     1|   Ankit|       Rohit|             Mohit|
|     2|   Mohit|       Mudit|              Agam|
|     3|   Vikas|       Rohit|             Mohit|
|     4|   Rohit|       Mohit|             Mudit|
|     5|   Mudit|        Agam|             Mohit|
|     6|    Agam|       Mohit|             Mudit|
|     7|  Sanjay|       Mohit|             Mudit|
|     8|  Ashish|       Mohit|             Mudit|
|     9|  Mukesh|        Agam|             Mohit|
|    10|  Rakesh|        Agam|             Mohit|
+------+--------+------------+------------------+

